(sqlite-datatypes)=
# Data types in SQLite

In [1]:
%LOAD ../data/employees.db

De datatypes in SQLite zijn eenvoudiger dan die in andere SQL-database-systemen (zie {ref}`mysql-datatypes`).
Waar je in MySQL de (maximale) grootte op kunt geven van getallen of van tekst-waarden, gebruikt SQLite een flexibele maar efficiënte manier om de gegevens (compact) op te slaan.
SQLite heeft de volgende datatypes:

* INTEGER
* REAL
* TEXT
* BLOB
* NULL

SQLite heeft geen datum/tijd-type: voor datum-gegevens gebruik je tekst-waarden of getallen.
(Zie: {ref}`mysql-datumtijd`.)

Voor een uitgebreide beschrijving van SQLite datatypes, zie: https://www.sqlite.org/datatype3.html

## Getallen: INTEGER en REAL

Er zijn twee soorten numerieke gegevens:

* gehele getallen (`INTEGER` of `INT`) - afhankelijk van de waarde,  0, 1, 2, 3, 4, 6, of 8 bytes
* "zwevende punt" (floating point) getallen (`REAL`) - 8 bytes "double" IEEE

Let op dat (zoals in de meestel computerprogramma's) een punt gebruikt wordt waar in Nederland een komma gebruikelijk is ("zwevende komma") - dus 3.1415926 in plaats van 3,1415926.

## Alfanumeriek: TEXT

Er is maar één type voor alfanumerieke gegevens: `TEXT`.
Dit heeft een variabele lengte, maximaal ca. 1-2 GByte, afhankelijk van de implementatie.

## BLOB

Een BLOB is een *Binary Large OBject*, een reeks bytes van variabele lengte. Dit gebruik je bijvoorbeeld voor het opslaan van plaatjes of geluidsbestanden.
De maximale lengte van een BLOB is net als die van TEXT, ca. 1-2 GByte, afhankelijk van de implementatie.

## CAST: type-conversie

In principe zet SQL een getal automatisch (impliciet) om naar een TEXT-waarde, als er mee gerekend wordt; of omgekeerd, van getal naar TEXT, als er een TEXT-operatie op uitgevoerd wordt.
Maar het is beter om deze omzetting altijd duidelijk (expliciet) aan te geven, met behulp van de `CAST` constructie.

* syntax: `CAST(expression AS type-name)`
* voorbeeld: `CAST(salary AS TEXT)`

In het voorbeeld hieronder gebeurt een impliciete omzetting van TEXT naar INT, zoals je ziet aan het type van de `bonus` kolom:

In [4]:
SELECT last_name, typeof(bonus), bonus,  bonus + 100
FROM employees
WHERE bonus IS NOT NULL;

last_name,typeof(bonus),bonus,bonus + 100
Zlotkey,text,1500,1600
Abel,text,1700,1800
Taylor,text,1250,1350


Het is beter om de omzetting (conversie) expliciet te maken:

In [5]:
SELECT last_name, typeof(bonus), bonus,  CAST(bonus as INT) + 100
FROM employees
WHERE bonus IS NOT NULL;

last_name,typeof(bonus),bonus,CAST(bonus as INT) + 100
Zlotkey,text,1500,1600
Abel,text,1700,1800
Taylor,text,1250,1350


Let op: in SQLite resulteert het afronden van een REAL in een afgeronde REAL waarde, niet in een INT waarde.

In [7]:
SELECT round(3.14);

round(3.14)
3.0


In [8]:
SELECT CAST(round(3.14) AS INT);

CAST(round(3.14) AS INT)
3


## `format` functie: NUMBER naar TEXT

Voor het omzetten van een getal (INT of REAL) naar TEXT-formaat kun je in SQLite de `format` functie gebruiken.
Deze heet ook wel `printf`, en werkt als de C-functie `printf`:

* de eerste parameter is een format-string; hierin geef je aan op welke positie het getal ingevuld moet worden, en in welke notatie (bijvoorbeeld: aantal cijfers na de decimale punt).
* je kunt allerlei tekst rond het getal plaatsen; bijvoorbeeld een `$`-teken voor een bedrag.
* de volgende parameter is het getal dat in de format-string ingevuld moet worden.
* je kunt ook andere waarden dan getallen invullen (TEXT bijvoorbeeld); en je kunt meerdere waarden invullen in de format-string.
* voor een uitgebreide beschrijving, zie: https://www.sqlite.org/draft/printf.html

Voorbeeld:

* `format('$ %d', round(3.5))` (d: geheel getal in decimale notatie
* `format('pi = %.3f', 3.1415926)` (f: floating point notatie, 3 cijfers na de `.`)

In [52]:
SELECT format('$ %d',123) as bedrag;

bedrag
$ 123


In [42]:
SELECT format('pi = %.3f', 3.1415926) as PI;

PI
pi = 3.142


Merk op dat in dit laatste geval het getal afgerond wordt.

### Geen CONVERT

SQLite heeft geen `CONVERT`-functie zoals MySQL. Je kunt het gewenste resultaat krijgen met behulp van `CAST`.